## Relu (활성화 함수)

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [5]:
device = 'cuda'

random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [6]:
# 파라미터
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [7]:
# 데이터 가져오기
mnist_train = dsets.MNIST(root='./MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='./MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

9920512it [00:01, 6594984.47it/s]                              


Extracting ./MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST_data/MNIST/raw


32768it [00:00, 82002.88it/s]            


Extracting ./MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST_data/MNIST/raw


1654784it [00:01, 1585101.29it/s]                            


Extracting ./MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST_data/MNIST/raw


8192it [00:00, 29707.00it/s]            

Extracting ./MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST_data/MNIST/raw
Processing...
Done!


In [8]:
# dataset loader
# 배치 사이즈만큼 이미지 데이터들이 묶여있음
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                         batch_size = batch_size,
                                         shuffle=True,
                                         drop_last = True)

In [10]:
# nn layers
linear1 = torch.nn.Linear(784, 256, bias = True)
linear2 = torch.nn.Linear(256,  256, bias = True)
linear3 = torch.nn.Linear(256,10, bias = True)

relu = torch.nn.ReLU()

In [12]:
# 가중치 초기화
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[-0.6645,  0.6646,  0.7159,  ..., -0.3040, -0.8945, -0.7977],
        [-1.2045,  2.4545, -1.8073,  ..., -1.7139, -0.0451,  0.3864],
        [ 0.3414,  0.3114, -1.9218,  ..., -0.4525, -0.6849,  0.9663],
        ...,
        [ 1.3915, -0.4048,  0.2338,  ..., -0.8257, -0.8397,  1.9816],
        [ 1.0414,  0.2130, -0.0417,  ...,  1.7541, -0.6454, -0.0821],
        [-0.1162, -1.2692,  0.8201,  ..., -0.8303, -0.5022,  0.0583]],
       requires_grad=True)

In [13]:
# 모델 생성
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [14]:
# cost, loss, optimizer 정의
criterion = torch.nn.CrossEntropyLoss().to(device) # 소프트맥스(출력층 활성화함수) 함수가 내장됨
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [27]:
print(data_loader.dataset)
for x,label in data_loader :
    print(x.size())
    print(label.size())
    break

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./MNIST_data/
    Split: Train
    StandardTransform
Transform: ToTensor()
torch.Size([100, 1, 28, 28])
torch.Size([100])


In [28]:
total_batch = len(data_loader)
print(total_batch)

600


In [31]:
# training
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X,Y in data_loader:
        X = X.view(-1,28*28).to(device) # reshape. 한 이미지에는 784개의 픽셀이 존재함
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch # 배치묶음별 평균 cost 저장
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 128.263015747
Epoch: 0002 cost = 36.130977631
Epoch: 0003 cost = 23.104619980
Epoch: 0004 cost = 15.961974144
Epoch: 0005 cost = 11.522852898
Epoch: 0006 cost = 8.607193947
Epoch: 0007 cost = 6.406140327
Epoch: 0008 cost = 4.722168446
Epoch: 0009 cost = 3.563386917
Epoch: 0010 cost = 2.695284605
Epoch: 0011 cost = 2.082398891
Epoch: 0012 cost = 1.677250147
Epoch: 0013 cost = 1.202304244
Epoch: 0014 cost = 0.960374713
Epoch: 0015 cost = 0.861065149
Learning finished


In [36]:
# test
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1,28*28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())
    
    # 이미지 하나 예측
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28*28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r+1].to(device)
    
    print(Y_single_data)
    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9458999633789062
tensor([8], device='cuda:0')
Label:  8
Prediction:  8
